In [29]:
print("hello World!")

hello World!


In [30]:
%pwd

'/workspaces/End-to-End-Medical-Chatbot-AI-2-'

In [31]:
import os
os.chdir("../")

In [32]:
%pwd

'/workspaces'

In [2]:
import sys, pkgutil
print(sys.executable)
print("langchain" in {m.name for m in pkgutil.iter_modules()})


/opt/conda/envs/medbot/bin/python
True


In [1]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [3]:
# Extarction of data from the PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls = PyPDFLoader)
    documents = loader.load()

    return documents

In [4]:
extracted_data  = load_pdf_file(data='/workspaces/End-to-End-Medical-Chatbot-AI-2-/Data/')
print(f"Loaded {len(extracted_data)} documents")


incorrect startxref pointer(1)
parsing for Object Streams


Loaded 2277 documents


In [5]:
extracted_data

[Document(metadata={'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'moddate': '2017-05-01T10:37:35-07:00', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'source': '/workspaces/End-to-End-Medical-Chatbot-AI-2-/Data/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'total_pages': 759, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'moddate': '2017-05-01T10:37:35-07:00', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'source': '/workspaces/End-to-End-Medical-Chatbot-AI-2-/Data/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'total_pages': 759, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metadata={'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'moddate': '2017-05-01T10:37:35-07:00', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'sou

In [6]:
# Split the Data into chunks as described before
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter (chunk_size=500 , chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
# Check if data is veing actually loaded
print(f"Documents loaded: {len(extracted_data)}")
if len(extracted_data) > 0:
    print(extracted_data[0].page_content[:500])  # first 500 chars


Documents loaded: 759



In [8]:
# Execution
text_chunks = text_split(extracted_data)
print("Length of text chunks :" , len(text_chunks))

Length of text chunks : 0


In [9]:
if extracted_data:
    print(repr(extracted_data[0].page_content[:300]))
# I need OCR:)

''


In [10]:
#OCR
# Robust PDF loader + OCR + splitter (LangChain-compatible)

import os
from typing import List
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# --- loaders ---
from langchain_community.document_loaders import PyPDFLoader

# OCR / rendering
import fitz  # PyMuPDF
import pytesseract
from PIL import Image


def _ocr_pdf_per_page(pdf_path: str, zoom: float = 2.0) -> List[Document]:
    """OCR each page -> list[Document] (one Document per page)."""
    docs: List[Document] = []
    with fitz.open(pdf_path) as pdf:
        mat = fitz.Matrix(zoom, zoom)  # 2.0 ~144dpi; bump to 3.0 for tougher scans
        for i, page in enumerate(pdf):
            pix = page.get_pixmap(matrix=mat)
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            txt = pytesseract.image_to_string(img, config="--psm 6")  # block of text mode
            txt = (txt or "").strip()
            if txt:
                docs.append(
                    Document(
                        page_content=txt,
                        metadata={"source": pdf_path, "page": i + 1, "loader": "ocr"},
                    )
                )
    return docs


def load_pdf_file_ocr_aware(data_dir: str) -> List[Document]:
    """
    Load PDFs from a folder.
    1) Try PyPDFLoader (text-layer PDFs)
    2) If empty or pages have no text, fallback to OCR per page
    Returns a list[Document] with non-empty page_content.
    """
    if not os.path.isdir(data_dir):
        raise FileNotFoundError(f"Folder not found: {data_dir}")

    pdf_paths = sorted(
        os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.lower().endswith(".pdf")
    )
    if not pdf_paths:
        print(f"No PDFs found in {data_dir}")
        return []

    all_docs: List[Document] = []
    for path in pdf_paths:
        # 1) Try normal text extraction
        try:
            docs = PyPDFLoader(path).load()
        except Exception as e:
            print(f"[PyPDFLoader] Skipped {os.path.basename(path)}: {e}")
            docs = []

        # Keep only pages with non-empty text
        docs = [d for d in docs if (d.page_content or "").strip()]

        # 2) If no text pages, OCR fallback
        if not docs:
            ocr_docs = _ocr_pdf_per_page(path, zoom=2.0)
            if not ocr_docs:
                print(f"[WARN] No extractable text in {os.path.basename(path)} (even after OCR).")
            all_docs.extend(ocr_docs)
        else:
            all_docs.extend(docs)

    return all_docs


def text_split(documents: List[Document], chunk_size: int = 1000, chunk_overlap: int = 200):
    """
    Split Document objects into chunks using LangChain's RecursiveCharacterTextSplitter.
    Guaranteed to return [] only if there was truly no text after OCR.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(documents)


# ========= EXECUTION =========
DATA_DIR = "/workspaces/End-to-End-Medical-Chatbot-AI-2-/Data/"  # adjust if needed
extracted_data = load_pdf_file_ocr_aware(DATA_DIR)

print("Docs loaded (pages with text):", len(extracted_data))
if extracted_data:
    print("Preview:", repr(extracted_data[0].page_content[:300]))

text_chunks = text_split(extracted_data)
print("Length of text chunks:", len(text_chunks))
if text_chunks:
    print("First chunk preview:\n", text_chunks[0].page_content[:400])


incorrect startxref pointer(1)
parsing for Object Streams


MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: 

In [11]:
! pip install pymupdf
! pip install pytesseract


In [14]:
import fitz
path = '/workspaces/End-to-End-Medical-Chatbot-AI-2-/Data/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'

pdf = fitz.open(path)
print(len(pdf), "pages")


759 pages


In [16]:
for i, page in enumerate(pdf):
    try:
        pix = page.get_pixmap()
    except Exception as e:
        print(f"Skipping page {i+1} due to error: {e}")
        continue


MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: library error: zlib error: 

In [17]:
import fitz
path = "/workspaces/End-to-End-Medical-Chatbot-AI-2-/Data/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf"
doc = fitz.open(path)  # If this line raises the zlib error, proceed to step 3
print(len(doc), "pages")


759 pages


In [18]:
import fitz
path = "/workspaces/End-to-End-Medical-Chatbot-AI-2-/Data/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND_fixed.pdf"  # or _gs.pdf
doc = fitz.open(path)
print(len(doc), "pages")


759 pages


In [5]:
import fitz
doc = fitz.open("/workspaces/End-to-End-Medical-Chatbot-AI-2-/Data/The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_id_.pdf")
print("Pages:", len(doc))


FileNotFoundError: no such file: '/workspaces/End-to-End-Medical-Chatbot-AI-2-/Data/The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_id_.pdf'